In [207]:
from os.path import join as pjoin
import tqdm
import math
import pandas as pd
import numpy as np
import os
import torchvision
import time 
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import sklearn
from PIL import Image
import torch.optim as optim
from sklearn.model_selection import train_test_split
import gc
import cv2
import torchmetrics
import matplotlib.pyplot as plt
from  torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassPrecision, MulticlassRecall
from torchmetrics.classification import F1Score, Recall, Precision

In [208]:
label_map = {
             'abraham_grampa_simpson': 0,
             'agnes_skinner': 1,
             'apu_nahasapeemapetilon': 2,
             'barney_gumble': 3,
             'bart_simpson': 4,
             'carl_carlson': 5,
             'charles_montgomery_burns': 6,
             'chief_wiggum': 7,
             'cletus_spuckler': 8,
             'comic_book_guy': 9,
             'disco_stu': 10,
             'edna_krabappel': 11,
             'fat_tony': 12,
             'gil': 13,
             'groundskeeper_willie': 14,
             'homer_simpson': 15,
             'kent_brockman': 16,
             'krusty_the_clown': 17,
             'lenny_leonard': 18,
             'lionel_hutz': 19,
             'lisa_simpson': 20,
             'maggie_simpson': 21,
             'marge_simpson': 22,
             'martin_prince': 23,
             'mayor_quimby': 24,
             'milhouse_van_houten': 25,
             'miss_hoover': 26,
             'moe_szyslak': 27,
             'ned_flanders': 28,
             'nelson_muntz': 29,
             'otto_mann': 30,
             'patty_bouvier': 31,
             'principal_skinner': 32,
             'professor_john_frink': 33,
             'rainier_wolfcastle': 34,
             'ralph_wiggum': 35,
             'selma_bouvier': 36,
             'sideshow_bob': 37,
             'sideshow_mel': 38,
             'snake_jailbird': 39,
             'troy_mcclure': 40,
             'waylon_smithers': 41
             }

In [209]:
#Penalty
from pathlib import Path
dict_num_for_img = {}
full_n = 0
folder = "/kaggle/input/simpson-norm/simpson/train"
for name, i in label_map.items():
    local_fold = Path(folder + "/" + name)
    sum_img = sum(1 for x in local_fold.iterdir())
    dict_num_for_img[name] = sum_img 
    full_n += sum_img

class_weight = []
for key, num in label_map.items():
    class_weight.append(full_n/dict_num_for_img[key])

weights = []
for i in range(len(class_weight)):
    weights.append(math.log(sum(class_weight)/class_weight[i]))

weights = torch.tensor(list(weights), dtype=torch.float)
print(weights)

tensor([6.5524, 3.4733, 6.1702, 4.3991, 6.9376, 4.3206, 6.8199, 6.6293, 3.5858,
        5.8863, 2.1336, 5.8604, 3.0315, 3.0315, 4.5315, 7.4526, 5.9463, 6.8307,
        5.4722, 1.6816, 6.9465, 4.5877, 6.8988, 3.9984, 5.2410, 6.7195, 2.5689,
        7.0164, 7.0178, 5.6162, 3.2014, 4.0123, 6.8207, 3.9101, 3.5423, 4.2243,
        4.3704, 6.5122, 3.4246, 3.7430, 1.8151, 4.9342])


In [210]:
learning_rate = 0.01#0.01
weight_decay_adam = 0.005#0.005
weight_decay = 0.005
batch_size = 128
momentum = 0.9
num_epochs = 100
drop_p = 0.25
L2_enable = False
L1_enable = False
mean = [0.5881, 0.6786, 0.6122]
std = [0.2290, 0.2239, 0.2251]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
weights = weights.to(device)
criterion = nn.CrossEntropyLoss(weight = weights, reduction='mean')

In [211]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees = 90),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])
#ColorJitter
dataset_train = torchvision.datasets.ImageFolder(root='/kaggle/input/simpson-norm/simpson/train', transform=transform)
dataset_test  = torchvision.datasets.ImageFolder(root='/kaggle/input/simpson-norm/simpson/test' , transform=transform)

num_train = len(dataset_train)
indices = list(range(num_train))

split = int(np.floor(0.8 * num_train))

np.random.seed(np.random.randint(0, 10000))
np.random.shuffle(indices)

train_idx, valid_idx = indices[:split], indices[split:]

train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(valid_idx)

In [212]:
train_loader = torch.utils.data.DataLoader(dataset_train, sampler = train_sampler, batch_size=batch_size, 
                                         num_workers=0, drop_last=True)

valid_loader = torch.utils.data.DataLoader(dataset_train, sampler = valid_sampler, batch_size=batch_size, 
                                         num_workers=0, drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, 
                                         num_workers=0, drop_last=False)

In [213]:
class SimpsonsCNN(nn.Module):
    def __init__(self):
        super(SimpsonsCNN, self).__init__()

        self.layer1 = nn.Sequential(nn.Conv2d( 3, 32, kernel_size=3, stride=1, padding=1), 
                                    nn.Dropout(drop_p), nn.BatchNorm2d(32), nn.ELU())
        
        self.layer2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.layer3 = nn.Sequential(nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1), 
                                    nn.Dropout(drop_p), nn.BatchNorm2d(32), nn.ELU(),
                                    nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1), 
                                    nn.Dropout(drop_p), nn.BatchNorm2d(32), nn.ELU())
        
        self.layer4 = nn.MaxPool2d(kernel_size=2, stride=2) 

        self.layer5 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), 
                                    nn.Dropout(drop_p), nn.BatchNorm2d(64), nn.ELU())

        self.layer6 = nn.MaxPool2d(kernel_size=2, stride=2) 
        
        self.layer7 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), 
                                    nn.Dropout(drop_p), nn.BatchNorm2d(64), nn.ELU(),
                                    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), 
                                    nn.Dropout(drop_p), nn.BatchNorm2d(64), nn.ELU())
        
        self.layer8 = nn.MaxPool2d(kernel_size=2, stride=2)       
        
        self.layer9 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), 
                                    nn.Dropout(drop_p), nn.BatchNorm2d(128), nn.ELU())
        
        self.layer10 = nn.MaxPool2d(kernel_size=2, stride=2) 
        
        self.layer11 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), 
                                    nn.Dropout(drop_p), nn.BatchNorm2d(256), nn.ELU())
        
        self.layer12 = nn.MaxPool2d(kernel_size=2, stride=2) 
        
        self.fc1 = nn.Linear( 1*16*256,1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear( 512,  42)
        self.drop = nn.Dropout(drop_p)
        
    def forward(self, x):
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = x + self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = x + self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        
        x = x.view(-1, 1*16*256)
        x = F.elu(self.drop(self.fc1(x)))
        x = F.elu(self.drop(self.fc2(x)))
        x = self.fc3(x)

        return x

In [214]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def valid_map(model, loader):
    classes = label_map
    classes_amount = len(label_map)
    recall = MulticlassRecall(num_classes=classes_amount, average=None)
    precision = MulticlassPrecision(num_classes=classes_amount, average=None)
    recall.to(device)
    precision.to(device)
    valid_acc = 0
    with torch.no_grad():
        my_loss = 0
        model.eval()
        correct = 0
        total = 0
        for sample in loader:
            images, labels = sample[0], sample[1]
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            recall.update(outputs, labels)
            precision.update(outputs, labels)
            _, pred = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            my_loss += criterion(outputs, labels).item()
        valid_acc = 100 * correct / total
        avr_loss = my_loss/len(loader)
    val_rec = sum(recall.compute())/len(list(label_map))
    val_prec = sum(precision.compute())/len(list(label_map))
    precision = val_prec.item()
    recall = val_rec.item()
    f1 = 2 * precision * recall / (precision + recall)
    return avr_loss, valid_acc, recall, precision, f1

In [215]:
model = SimpsonsCNN()

In [216]:
model = model.to(device)
criterion = criterion.to(device)
print(count_parameters(model))

5223786


In [217]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay = weight_decay_adam)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum) 
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scaler = torch.cuda.amp.GradScaler()
torch.cuda.empty_cache()
gc.collect()
list_train_loss = []
list_valid_loss = []

In [ ]:
print(f'-------------------------------------Learning begin, total of {num_epochs} epochs-----------------------------------\n')
for epoch in range(num_epochs):
    correct, total = 0, 0

    model.train()
    for i, sample in enumerate(train_loader):
        img, labels = sample[0], sample[1]
        img = img.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast(enabled=True):
            output = model(img)
            loss = criterion(output, labels)
            _, pred = torch.max(output.data, 1)
        
        if L2_enable:
            tens = (0.5 * weight_decay * sum(p.pow(2.0).sum() for p in model.parameters()))
            l2_reg = (tens.clone().detach().to(device))
            for name, param in model.named_parameters():
                if 'weight' in name:
                    l2_reg = l2_reg + torch.norm(param)
            loss += weight_decay * l2_reg
        
        if L1_enable:
            tens = (0.5 * weight_decay * sum(p.abs().sum() for p in model.parameters()))
            l1_reg = (tens.clone().detach().to(device))
            for name, param in model.named_parameters():
                if 'weight' in name:
                    l1_reg = l1_reg + torch.norm(param)
            loss += weight_decay * l1_reg
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    
        correct += (pred == labels).sum().item()
        total += labels.size(0)
        
        if (i + 1) % 20 == 0:
            train_acc = 100 * correct / total
            correct, total = 0, 0
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], ', end = '')
            print(f'Loss: {loss.item():.4f}, Acc: {train_acc:.2f}%')
            list_train_loss.append(loss.item())
        torch.cuda.empty_cache()
        gc.collect()
        
    avr_loss, val_acc, val_rec, val_prec, val_f1 = valid_map(model, valid_loader)
    list_valid_loss.append(avr_loss)
    print(f'\n--------------------------------------------Result of Epoch {epoch}-------------------------------------------')
    print(f'Valid_Loss: {avr_loss:.4f}, Valid_accuracy: {val_acc:.2f}%, Valid Recall: {100*val_rec:.2f}%, ', end = '')
    print(f'Valid Precision: {100*val_prec:.2f}%, Valid F1: {100*val_f1:.2f}%')
    print(f'---------------------------------------------------------------------------------------------------------\n')
        
    if val_acc > 85:
        break
    scheduler.step()

-------------------------------------Learning begin, total of 100 epochs-----------------------------------

Epoch [1/100], Step [20/130], Loss: 8.0358, Acc: 5.98%
Epoch [1/100], Step [40/130], Loss: 4.6945, Acc: 6.33%


In [ ]:
print(f'-------------------------------------------Learning complete------------------------------------------\n')
avr_loss, test_acc, test_rec, test_prec, test_f1 = valid_map(model, test_loader)
print(f'\n--------------------------------------------Result of Testing-------------------------------------------')
print(f'Test_Loss: {avr_loss:.4f}, Test_accuracy: {test_acc:.2f}%, Test Recall: {100*test_rec:.2f}%, ', end = '')
print(f'Test Precision: {100*test_prec:.2f}%, Test F1: {100*test_f1:.2f}%')
print(f'---------------------------------------------------------------------------------------------------------\n')

In [ ]:
weights = []
for name, param in model.named_parameters():
    if 'weight' in name:
        weights += param.data.cpu().numpy().flatten().tolist()

weights = np.array(weights)

# Построение гистограммы
plt.hist(weights, bins=80, range = (-0.02, 0.02))
plt.xlabel('Weight values')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#torch.save(model.state_dict(), 'my_model.pt')
#model.load_state_dict(torch.load('my_model.pt', map_location=device))

In [ ]:
fig = plt.figure(figsize=(10, 10))

plt.plot(list_train_loss, label='train')
plt.plot(list_valid_loss, label='valid')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('Loss')

In [ ]:
loader = test_loader
classes = label_map
classes_amount = len(label_map)
recall = MulticlassRecall(num_classes=classes_amount, average=None)
precision = MulticlassPrecision(num_classes=classes_amount, average=None)
recall.to(device)
precision.to(device)
valid_acc = 0
with torch.no_grad():
        my_loss = 0
        model.eval()
        correct = 0
        total = 0
        for sample in loader:
            images, labels = sample[0], sample[1]
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            recall.update(outputs, labels)
            precision.update(outputs, labels)
            _, pred = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            my_loss += criterion(outputs, labels).item()
        valid_acc = 100 * correct / total
        avr_loss = my_loss/len(loader)
        
recall_class = {classname: val.item()
                for classname, val in zip(label_map, recall.compute())}
precision_class = {classname: val.item()
                   for classname, val in zip(label_map,precision.compute())}
metrics_per_class = {
    "recall": recall_class, "precision": precision_class}

fig, axes = plt.subplots(1, 2, figsize=(25, 5))
for (metricName, mVal), ax in zip(metrics_per_class.items(), axes):
    plt.sca(ax)
    plt.bar(mVal.keys(), mVal.values())
    plt.ylabel(metricName)
    plt.xticks(rotation=90)
    plt.grid(axis='y')
    plt.yticks(ticks=np.arange(0, 1.01, 0.05))
plt.show()

In [ ]:
#model = torchvision.models.resnet50(pretrained=True)
#for param in model.parameters():
#    param.requires_grad = False

#model.fc = nn.Sequential(
#    nn.Linear(1024, 128),
#    nn.ReLU(inplace=True),
#    nn.Linear(128, 42))

#import time 
#start = time.time()
#end = time.time()
#print(end - start)

In [ ]:
#def batch_mean_and_std(loader):
#    cnt = 0
#    fst_moment = torch.empty(3)
#    snd_moment = torch.empty(3)

#    for images, _ in loader:
#        b, c, h, w = images.shape
#        nb_pixels = b * h * w
#        sum_ = torch.sum(images, dim=[0, 2, 3])
#        sum_of_square = torch.sum(images ** 2,
#                                  dim=[0, 2, 3])
#        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
#        snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)
#        cnt += nb_pixels

#    mean, std = fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)        
#    return mean,std
  
#mean, std = batch_mean_and_std(train_loader)
#print(mean, std)
#mean = [0.5881, 0.6786, 0.6122], std = [0.2290, 0.2239, 0.2251]

In [ ]:
#for name, layer in model.named_children():
#    print(name, layer)

In [ ]:
#path_for_test = 'simpson/archive_test'
#names = []
#for dirs, folder, files in os.walk(path_for_test):
#     for img in files:
#         ind = '_'.join(os.path.splitext(os.path.basename(path_for_test + '/' + img))[0].split('_')[:-1])
#         names.append(ind)
#names = list(set(names))
#print(len(names))
#folder_path = 'simpson'
#os.mkdir(folder_path+'/test')
#folder_path = folder_path + '/test'
#for folder in names:
#     if not os.path.exists(folder_path + '/' + folder):
#         os.mkdir(folder_path + '/' + folder)
#list_of_names = os.listdir(folder_path)
#print(list_of_names)
#for dirs, folder, files in os.walk(path_for_test):
#     for img in files:
#         ind = '_'.join(os.path.splitext(os.path.basename(path_for_test + '/' + img))[0].split('_')[:-1])
#         for name in list_of_names:
#             if ind == name:
#                 shutil.move(path_for_test + '/' + img, folder_path + '/' + name)

In [ ]:
#n_classes = 42
#class_counts = [0] * n_classes
#for _, target in train_loader.dataset:
#    class_counts[target] += 1
#n_samples = len(train_loader.dataset)
#weights = torch.tensor([n_samples / (n_classes * class_counts[i]) for i in range(n_classes)], dtype=torch.float)

